<a href="https://colab.research.google.com/github/rsv141295022/Coursera/blob/main/1_Data_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import datetime

from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

0it [00:00, ?it/s]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Smart Meter and Weather datasets

we select data between 7th Feb 2020 to 11th Aug 2021 becuase the weather dataset contains farthest data in 7th Feb 2020

In [6]:
# Import Smart Meter dataframe
ls = "/content/drive/My Drive/Colab Notebooks/IS project/smart_meters/"
file_names = ['Bundit.csv', 'Erdi.csv', 'Female_2.csv', 'Female_4.csv', 'Female_6.csv', 'Female_8.csv', 
             'Male_3.csv', 'Male_4.csv', 'Male_5.csv', 'Male_7.csv', 'Research_health_cmu.csv', 'Research_health_suandok.csv',
             'Subject.csv', 'Tech_service.csv', 'Tech.csv', 'Animal.csv', 'Data_science.csv', 'Dean_education.csv', 'Library.csv', 
              'Mechanical.csv', '30years.csv', 'Econ.csv', 'Acc_ba.csv', 'resource_argo.csv', 'Physics_excellent.csv']
raw = {}
for x in file_names:
    raw["{}".format(x[:-4])] = pd.read_csv(ls + x, usecols=["timeIn", "energyConsumtion"], parse_dates=[0],
                   infer_datetime_format=True, index_col='timeIn').loc['2020-02-07':'2021-08-11',:]

# Import weather dataframe
weather = pd.read_excel(ls+'weather.xlsx')
weather['timestamp'] = pd.to_datetime(weather.timestamp, infer_datetime_format=True)  
weather = weather.sort_values(by='timestamp')

# Create the Integrated dataframe
- Creating a dataframe with **rows of 15-mins interval** from date 7th Feb 2020 to 12th Aug 2021 and **columns of every building name and weather dataset column name** (pm2.5, pm10, temp, humid)

- Then put NaN values into every single value in the dataframe

- Because we can find weather data furthest day back in 7th Feb 2020)




In [7]:
# create dataframe of datetime from 07-Feb-2020 to 06-July-2021
df_integrated = pd.DataFrame({"Time": pd.date_range(start='2020-02-07', end='2021-08-12', freq='15min')})

# create cols of smart meter and weather dataset
df_integrated[['pm10', 'pm2.5', 'temp', 'humid']] = np.nan
df_integrated[list(raw.keys())] = np.nan
#df_ideal.set_index('Time', inplace=True)

df_integrated.head()

,Time,pm10,pm2.5,temp,humid,Bundit,Erdi,Female_2,Female_4,Female_6,Female_8,Male_3,Male_4,Male_5,Male_7,Research_health_cmu,Research_health_suandok,Subject,Tech_service,Tech,Animal,Data_science,Dean_education,Library,Mechanical,30years,Econ,Acc_ba,resource_argo,Physics_excellent
0,2020-02-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-07 00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-07 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-07 00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-07 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Fill Weather data into Integrated dataframe

To fill data into the integrated dataframe that we created above, I write a function to find the nearset time in weather dataset by putting time form the integrated dataframe

In [9]:
def fill_nearest_weather(time):
  less_15_min = abs(weather.timestamp - time) < datetime.timedelta(minutes=15)

  if less_15_min.any():
    return weather[weather.timestamp == min(weather.timestamp[less_15_min], key=lambda x: abs(x - time))][col].values[0]

  else: 
    return np.nan

In [10]:
for col in ['pm10', 'pm2.5', 'temp', 'humid']:
  df_integrated[col] = df_integrated['Time'].progress_apply(lambda x: fill_nearest_weather(x))

  0%|          | 0/52993 [00:00<?, ?it/s]

  0%|          | 0/52993 [00:00<?, ?it/s]

  0%|          | 0/52993 [00:00<?, ?it/s]

  0%|          | 0/52993 [00:00<?, ?it/s]

In [11]:
df_integrated

,Time,pm10,pm2.5,temp,humid,Bundit,Erdi,Female_2,Female_4,Female_6,Female_8,Male_3,Male_4,Male_5,Male_7,Research_health_cmu,Research_health_suandok,Subject,Tech_service,Tech,Animal,Data_science,Dean_education,Library,Mechanical,30years,Econ,Acc_ba,resource_argo,Physics_excellent
0,2020-02-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-07 00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-07 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-07 00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-07 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52988,2021-08-11 23:00:00,4.0,3.0,31.0,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52989,2021-08-11 23:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52990,2021-08-11 23:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52991,2021-08-11 23:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Fill Smart Meter data into Integrated dataframe

To fill data into the integrated dataframe that we created above, I write a function to find the nearset time in smart meter dataset by putting time form the integrated dataframe

In [12]:
def fill_nearest_meter(time, data):

  less_15_min = abs(data.timeIn - time) < datetime.timedelta(minutes=15)
  
  if less_15_min.any():
    return data[data.timeIn == min(data.timeIn[less_15_min], key=lambda x: abs(x - time))]['energyConsumtion'].values[0]

  else: 
    return np.nan

In [13]:
for name, data in raw.items():
  data.reset_index(inplace=True)
  df_integrated[name] = df_integrated['Time'].progress_apply(lambda x: fill_nearest_meter(x, data))

  0%|          | 0/52993 [00:00<?, ?it/s]

  0%|          | 0/52993 [00:00<?, ?it/s]

  0%|          | 0/52993 [00:00<?, ?it/s]

  0%|          | 0/52993 [00:00<?, ?it/s]

  0%|          | 0/52993 [00:00<?, ?it/s]

...

  0%|          | 0/52993 [00:00<?, ?it/s]

  0%|          | 0/52993 [00:00<?, ?it/s]

  0%|          | 0/52993 [00:00<?, ?it/s]

  0%|          | 0/52993 [00:00<?, ?it/s]

  0%|          | 0/52993 [00:00<?, ?it/s]

In [16]:
df_integrated

,Time,pm10,pm2.5,temp,humid,Bundit,Erdi,Female_2,Female_4,Female_6,Female_8,Male_3,Male_4,Male_5,Male_7,Research_health_cmu,Research_health_suandok,Subject,Tech_service,Tech,Animal,Data_science,Dean_education,Library,Mechanical,30years,Econ,Acc_ba,resource_argo,Physics_excellent
0,2020-02-07 00:00:00,NaN,NaN,NaN,NaN,920899.780,333785.89000,718898.370,702512.000,437358.340,478461.890,890476.290,801161.280,NaN,683209.15,625372.03,4889358.34,4129253.63,158004.000,NaN,1.928849e+06,109467.53,593340.22000,3399406.080,204614.270,1084957.31,660636.74000,1250305.790,53724.340,NaN
1,2020-02-07 00:15:00,NaN,NaN,NaN,NaN,920904.320,333790.37000,718905.020,702512.000,437365.150,478468.540,890481.600,801170.180,NaN,683215.04,625376.32,4889370.11,4129271.81,158005.140,NaN,1.928864e+06,109467.75,593342.14000,3399411.200,204614.990,1084963.07,660640.00000,1250310.660,53724.820,NaN
2,2020-02-07 00:30:00,NaN,NaN,NaN,NaN,920908.800,333793.82000,718911.940,702517.440,437371.650,478475.140,890486.590,801178.110,NaN,683221.63,625380.48,4889382.91,4129287.42,158006.290,NaN,1.928882e+06,109467.99,593344.13000,3399416.320,204615.700,1084968.96,660643.20000,1250315.010,53725.290,NaN
3,2020-02-07 00:45:00,NaN,NaN,NaN,NaN,920913.150,333797.28000,718918.210,702528.060,437380.220,478481.250,890491.460,801186.500,NaN,683229.12,625384.58,4889395.20,4129304.58,158007.250,NaN,1.928905e+06,109468.19,593346.11000,3399421.180,204616.460,1084975.10,660646.40000,1250319.870,53725.760,NaN
4,2020-02-07 01:00:00,NaN,NaN,NaN,NaN,920917.570,333800.83000,718924.540,702532.610,437385.730,478486.980,890496.190,801186.500,NaN,683235.65,625388.67,4889406.98,4129321.73,158008.300,NaN,1.928923e+06,109468.40,593348.03000,3399426.560,204617.150,1084981.12,660649.54000,1250324.610,53726.220,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52988,2021-08-11 23:00:00,4.0,3.0,31.0,62.0,1408165.248,639286.11245,906575.424,907981.120,630285.888,739076.416,1129956.224,1038433.792,1258692.224,NaN,NaN,NaN,NaN,227974.832,NaN,3.659003e+06,NaN,955204.62947,4727628.288,329073.760,NaN,970575.90701,1830007.040,89628.328,11989.603
52989,2021-08-11 23:15:00,NaN,NaN,NaN,NaN,1408169.600,639290.36218,906578.944,907986.880,630290.752,739082.880,1129963.904,1038440.256,1258696.448,NaN,NaN,NaN,NaN,227975.520,NaN,3.659039e+06,NaN,955206.43688,4727633.920,329075.392,NaN,970578.12192,NaN,89628.896,11989.980
52990,2021-08-11 23:30:00,NaN,NaN,NaN,NaN,1408173.696,639294.39418,906582.976,907992.384,630295.488,739089.280,1129971.968,1038446.720,1258700.800,NaN,NaN,NaN,NaN,227976.192,NaN,3.659073e+06,NaN,955208.21848,4727640.064,329077.088,NaN,970580.29813,1830019.200,89629.472,11990.363
52991,2021-08-11 23:45:00,NaN,NaN,NaN,NaN,1408178.176,639298.88744,906586.624,907998.144,630300.160,739095.552,1129979.648,1038453.056,1258704.896,NaN,NaN,NaN,NaN,227976.896,NaN,3.659104e+06,NaN,955209.92805,4727645.696,329078.976,NaN,970582.38079,1830019.200,89629.848,11990.711


In [17]:
df_integrated.to_csv('final_integrated_dataset.csv')